In [1]:
import numpy as np
import pandas as pd

# df = pd.read_csv("../dataset/test.csv")
df = pd.read_csv("../dataset/sqli.csv",encoding='utf-16')
df

FileNotFoundError: [Errno 2] No such file or directory: './dataset/sqli.csv'

In [ ]:
## download the corpus of stopwords
import nltk
from nltk.corpus import stopwords 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ken/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Extract the feature of text

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# "min_df = 2" means  ignore the word that only appear one time in all documents
# "max_df = 0.7" means ignore the word taht appear in more than 70% of the documents
vectorizer = CountVectorizer( min_df=2, max_df=0.7, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(df['Sentence'].values.astype('U')).toarray()
transformed_posts=pd.DataFrame(posts)
# transformed_posts=pd.DataFrame(posts, columns=vectorizer.get_feature_names())

# Save the vectorizer by using serialization

In [ ]:
import pickle
with open("count_vectorizer.pk", "wb") as fin:
    pickle.dump(vectorizer.vocabulary_, fin)

In [ ]:
df=pd.concat([df,transformed_posts],axis=1)
df

,Sentence,Label,0,1,2,3,4
0,a,1,0,0,0,0,0
1,a',1,0,0,0,0,0
2,select * from users where id = '1' or @ @1 = ...,1,1,2,1,1,1
3,"select * from users where id = 1 or 1#"" ( un...",1,1,2,1,1,1
4,""" or pg_sleep ( __TIME__ ) --",1,0,0,0,0,0
5,create user name identified by pass123 tempora...,1,0,0,0,1,0
6,%29,1,0,0,0,0,0


In [ ]:
X=df[df.columns[2:]]
X

,0,1,2,3,4
0,0,0,0,0,0
1,0,0,0,0,0
2,1,2,1,1,1
3,1,2,1,1,1
4,0,0,0,0,0
5,0,0,0,1,0
6,0,0,0,0,0


In [ ]:
y=df['Label']
y

0    1
1    1
2    1
3    1
4    1
5    1
6    1
Name: Label, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

,0,1,2,3,4
5,0,0,0,1,0
2,1,2,1,1,1
4,0,0,0,0,0
3,1,2,1,1,1
6,0,0,0,0,0


# Using simple Neural Network

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

2021-07-27 18:06:01.106257: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-27 18:06:01.106299: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
input_dim = X_train.shape[1]    # number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              11264     
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 1025      
Total params: 16,715
Trainable params: 14,667
Non-trainable params: 2,048
________________________________________________

2021-07-27 18:06:02.849482: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-07-27 18:06:02.849539: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-27 18:06:02.849583: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (suisei): /proc/driver/nvidia/version does not exist


In [ ]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=15)

2021-07-27 18:06:03.547717: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-27 18:06:03.567633: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2600040000 Hz


Epoch 1/10
1/1 [==============================] - 21s 21s/step - loss: 1.0900 - accuracy: 0.4000 - val_loss: 0.6755 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 22ms/step - loss: 0.6543 - accuracy: 0.6000 - val_loss: 0.6582 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 21ms/step - loss: 0.9652 - accuracy: 0.4000 - val_loss: 0.6414 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 22ms/step - loss: 0.6337 - accuracy: 0.6000 - val_loss: 0.6244 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 19ms/step - loss: 0.6849 - accuracy: 0.4000 - val_loss: 0.6076 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 22ms/step - loss: 0.5675 - accuracy: 0.8000 - val_loss: 0.5910 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 24ms/step - loss: 0.6634 - accuracy: 0.6000 - val_loss: 0.5747 - val_accuracy: 1.0000
Epoch 8/10
1/1 [===========

In [ ]:
pred=model.predict(X_test)

for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

1.0

# Save the model

In [ ]:
model.save('sql_injection_detecting.h5', include_optimizer=False)

# How to predict for one string

In [ ]:
from keras.models import load_model
from sklearn.feature_extraction.text import CountVectorizer
import pickle

model = load_model('sql_injection_detecting.h5')
test_string = "select name from users;"

loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vectorizer.pk", "rb")))
test_one = loaded_vec.transform([test_string]).toarray()
print(test_one)
print(test_one.tolist())
test_pred = model.predict(test_one)
print(test_pred)

2021-07-28 18:38:35.519469: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-28 18:38:35.519512: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


2021-07-28 18:38:37.929868: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-07-28 18:38:37.929911: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-28 18:38:37.929946: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (suisei): /proc/driver/nvidia/version does not exist


AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [ ]:
# test_one = vectorizer.transform(['select name from users']).toarray()
# print(test_one)
# test_pred = model.predict(test_one)
# test_pred

[[0 1 0 1 0]]


array([[0.58624685]], dtype=float32)